In [2]:
import os
import csv

import rdflib

In [55]:
# Create a graph and set the namespaces.
g = rdflib.Graph()

# Create a dict of all the URIs.
iri = dict(prov=rdflib.URIRef("http://www.w3.org/ns/prov#"),
           dcat=rdflib.URIRef("http://www.w3.org/ns/dcat#"),
           dc=rdflib.URIRef("http://purl.org/dc/elements/1.1/"),
           dct=rdflib.URIRef("http://purl.org/dc/terms/"),
           dctypes=rdflib.URIRef("http://purl.org/dc/dcmitype/"),
           rdf=rdflib.URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
           rdfs=rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#"),
           xml=rdflib.URIRef("http://www.w3.org/XML/1998/namespace"),
           xsd=rdflib.URIRef("http://www.w3.org/2001/XMLSchema#"),
           foaf=rdflib.URIRef("http://xmlns.com/foaf/0.1/"),
           skos=rdflib.URIRef("http://www.w3.org/2004/02/skos/core#"),
           nidm=rdflib.URIRef("http://nidm.nidash.org/"),
           xnat=rdflib.URIRef("https://central.xnat.org/"),
           project=rdflib.URIRef("https://central.xnat.org/data/projects/"))

# Create a dict of Namespaces to use.
ns = {k:rdflib.Namespace(v) for k, v in iri.iteritems()}

# Bind namespaces to the graph.
for k, v in ns.iteritems():
    g.bind(k, v)

In [56]:
# Manually create the top level dataset descriptor for XNAT Central
g.add([iri.get('xnat'), rdflib.RDF['type'], ns.get('dctypes')['Dataset']])
g.add([iri.get('xnat'), ns.get('dct')['title'], rdflib.Literal('XNAT Central', lang='en')])
g.add([iri.get('xnat'), ns.get('dct')['description'], rdflib.Literal('The XNAT Central repository.', lang='en')])
g.add([iri.get('xnat'), ns.get('dct')['publisher'], rdflib.URIRef("https://central.xnat.org/")])

In [61]:
with open('xnat-central-projects.csv', 'rb') as fi:
    for row in csv.DictReader(fi):
        try:
            project_iri = ns.get('project')[row.get('id')]
            g.add([iri.get('xnat'), ns.get('dct')['hasPart'], project_iri])
            g.add([project_iri, rdflib.RDF['type'], ns.get('dctypes')['Dataset']])
            g.add([project_iri, ns.get('dct')['title'], rdflib.Literal(row.get('Title'), lang='en')])
            g.add([project_iri, ns.get('dct')['description'], rdflib.Literal(row.get('Description'), lang='en')])
            g.add([project_iri, ns.get('dct')['alternative'], rdflib.Literal(row.get('Running Title'), lang='en')])
            g.add([project_iri, ns.get('dcat')['keyword'], rdflib.Literal(row.get('Keywords'), lang='en')])
            g.add([project_iri, ns.get('xnat')['mrCount'], rdflib.Literal(row.get('MR Count'))])
            g.add([project_iri, ns.get('xnat')['petCount'], rdflib.Literal(row.get('PET Count'))])
            g.add([project_iri, ns.get('xnat')['ctCount'], rdflib.Literal(row.get('CT Count'))])
            g.add([project_iri, ns.get('xnat')['role'], rdflib.Literal(row.get('Role'))])
            g.add([project_iri, ns.get('dct')['rights'], rdflib.Literal(row.get('Accessibility'), lang='en')])
        except Exception as error:
            print("{}: {}".format(project_iri, error))
         

https://central.xnat.org/data/projects/CENTRAL_OASIS_LONG: 'utf8' codec can't decode byte 0x92 in position 449: invalid start byte


In [65]:
g.serialize('xnat-central.ttl', format='turtle')